In [1]:
#!pip install tensorflow-datasets
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models

physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

ModuleNotFoundError: No module named 'matplotlib'

# tf.keras.datasets 를 통해 데이터 들고오기

### 1. mnist

손글씨 그림으로 기존의 모델을 구현할 때 사용하였던 데이터들이다.

- 60000개의 28X28 크기의 이미지
- 10개의 class
- grayscale으로 채널의 크기가 1이다.



```python
    tf.keras.datasets.mnist.load_data(
        path='mnist.npz'
    )
```


### 2. fashion_mnist

zalando라는 패션 회사의 제품 이미지 데이터

- 60000개의 28X28 크기의 이미지
- 10개의 class

```python
    tf.keras.datasets.fashion_mnist.load_data()
```

패션 데이터의 라벨 및 자세한 정보는 아래의 링크에서 확인할 수 있다.  
https://github.com/zalandoresearch/fashion-mnist

### 3. cifar10

### 4. cifar100

물체 분류 데이터셋으로, 클래스의 갯수에 따라 10, 100으로 나뉜다.

- 50000장의 32X32의 컬러 이미지 데이터
- y의 값은 (num_samples, 1)의 shape를 가진다. 즉, 하나의 이미지마다 scalar값이 아니라 1의 길이를 갖는 배열이다.

자세한 데이터는 아래의 링크로 확인 가능하다.  
https://www.cs.toronto.edu/~kriz/cifar.html


# 로컬 디렉토리를 통한 데이터 셋 구성


# 데이터셋 클래스 객체에 대한 여러가지 전처리

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

https://www.microsoft.com/en-us/download/details.aspx?id=54765

# image_dataset_from_directory

```python
tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)
```

## returns

tf.data.Dataset의 타입으로 데이터셋을 구성하여 반환한다. 이때, 데이터셋은 아래의 파라미터들을 통해 구성된다.


### label_mode

1. int
> 카테고리의 값을 정수로 변환하여 전달한다.  
> sparse_categorical_crossentropy  
> label의 경우,(batch_size, )의 shape를 가진다.

2. categorical
> 카테고리의 값을 디렉토리의 이름으로 전달한다.-> one-hot encoding된 상태로 전달한다.  
> categorical_crossentropy  
> label의 경우,(batch_size, num_classes)의 shape를 가진다. 

3. binary
> 카테고리를 이진 값으로 변환하여 전달한다. -> float32으로 사용  
> binary_cross_entropy  
> label의 경우,(batch_size, 1)의 shape를 가진다.


### color_mode

1. 'rgb'
2. 'grayscale'



### batch_size
말그대로 배치 사이즈, 배치 사이즈란, 1회의 수행 동안에 사용할 데이터의 크기를 말한다.   
이 크기 만큼 한번 돌기 때문에 도는 횟수는 N / batch_size 이겠죠?


### validation_split
0,1 사이의 실수값으로 validation set을 구성할 것인가를 결정한다.

### subset
위의 validation_split을 사용한 경우, training, validation 두가지의 subset이 나오기 때문에 해당 파라미터로 구분한다.

### seed
난수를 생성하기 위해 필요한 seed값을 전달한다. 이 때, 동일한 조건의 training, validation을 사용하기 위해서 반드시 명시하여줘야한다.\



### interpolation
image resizing할 때, 픽셀값의 변화를 어떻게 적용할 것인가에 대한 정의


In [ ]:
# 이미지 저장시의 에러 처리
import os

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    # 이부분 반드시 변경이 필요하다.
    folder_path = os.path.join("pets", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname) # PetImages/Cat/63.jpg
        try:
            fobj = open(fpath, "rb")
            # fobj.peek(10)은 10바이트를 가져오는데, 여기서는 보통
            # 헤더를 읽기 위해 전체를 가져온다고 생각해도 무방합니다.
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()
        
        # JFIF의 헤더를 들고 있지 않으면 손상된 파일이므로 삭제    
        if not is_jfif:
            num_skipped += 1
            os.remove(fpath)
print(num_skipped)

## load_img  

image 파일을 PIL객체로 만들어준다. 

```
tf.keras.preprocessing.image.load_img(
    path, grayscale=False, color_mode="rgb", target_size=None, interpolation="nearest"
)
```

## PIL -> numpy array

from tensorflow.keras.preprocessing.image import img_to_array

# tf.data.Dataset

매우 큰 데이터에 대해서 입력의 파이프라인을 구축한다. 데이터 변환, 전처리등의 작업을 편하게 할 수 있도록 하는 API이다.


- batch size를 정의하게 되면, batch size만큼의 하나의 element로 저장된다. 정의하지 않아도 32가 기본적으로 사용된다.  


- cardinality, 차수를 통해 batch의 갯수를 확인할 수 있다.  


- dataset의 경우, for loop / as_numpy_iterator()를 사용하는 경우를 제외하고 모든 원소가 tensor이다.


- tf.py_function()을 이용하여 작업을 그래프화하여 하는 경우, 성능상의 이득을 얻을 수 있다.(최적화는 공부 범위 밖)

```python
#py_function으로 graph화하기
train_dataset.take(1).map(
    lambda images, labels : tf.py_function(batch_func, [images, labels], [tf.float32, tf.float32]))
```


## take()

- 원소하나를 들고온다. == batch 한 개를 들고온다고 생각하면 된다.


- BatchDataset -> TakeDataset으로 변환된다.



## apply()

각 batch size의 dataset에 대해서 함수를 적용한다.


## as_numpy_iterator()

numpy 배열로 변환한다.


## batch()

batch_size만큼 잘라낸다.

## flat_map

- batch의 개념없이 모든 원소에 대해서 직접 접근, flatten과정 수행 

## from_tensor_slices()

dictinary 형태, tuple형태의 데이터에 대해서 dataset을 구성하고자할 떄, 유용하게 사용할 수 있는 함수이다.


## tf.data.Dataset.zip()

데이터의 형태가 모두 tf.data.Dataset일 경우, zip()을 사용하여 하나의 dataset으로 바꾼다.

## enumerate()

python의 함수와 동일한 기능을 한다.

## map()

dataset의 원소마다 함수를 수행할 수 있도록 하는 함수

#### map 을 사용하는 이유

데이터를 로드하는 과정과 병렬처리를 최적화할 수 있기 때문에 사용하는 것이 좋다.

## reduce

모든 원소에 대해서 함수를 수행하여 단일 값으로 반환하는 함수

## shuffle

말 그대로 섞는다. 이 때, buffer_size를 인자로 전달해야하는데 이 값은 반드시 전체 sample size보다는 크거나 같아야한다.
shuffle의 대상의 크기를 정하는 것이므로, 대상이 되지 않는 sample에 대해서는 섞이지 않는 문제가 발생할 수 있다.

## 실제 데이터에 응용해보기

`image_dataset_from_directory`을 사용하여 이미지 데이터를 로드할 수 있다. 특정 디렉토리 내에 레이블에 따라 디렉토리로 구분되어있어야한다. 하지만 대부분의 망할 데이터는 편안한 형태로 제공되지 않는다. 이런 경우에는 어떻게 데이터 셋을 구현할 것인가에 대해 생각해보자.

### 목적
레이블마다 다른 디렉토리 경로의 데이터들을 어떻게 취합하여 사용할 것인가? 


### 문제

1. `image_dataset_from_directory`의 경우, 반드시 디렉토리로 레이블이 구분되어있어야한다. 이러한 제약조건을 이길 수 있는 방법은 없을까?
    > tf.data.Dataset을 레이블마다 구현하여 파이프라인을 커스터마이징하여 결합한다.
    
2. 이미지가 구분되어있을 때, 레이블링은 어떻게 할 것인가?
    > 동일 디렉토리가 아닌 경우, 디렉토리명으로 구분하여
    
3. 이미지가 구분되어있지 않을 때 레이블링은 어떻게 할 것인가? 
    > 동일 디렉토리에 존재하는 경우에는 이름으로 구분하여 


### 해결책

1. `tf.data.Dataset.list_files` 을 사용하여 file list를 load한다. (file load pipeline)
2. file name만 가진 dataset 이기 때문에 해당 file name을 통해 image를 load하는 파이프라인을 구축한다.
3. file name을 통해 해당 이미지가 어떤 레이블인지를 판단한다.


### 필요한 함수들


#####  `tf.io.read_file()`

input pipeline의 맨처음으로, filename을 인자로 받아 해당 파일의 컨텐츠를 받아온다.   
이 때, 파일의 format에 대한 제약조건은 존재하지 않는다.  
이미지의 경우, encoding된 'text' 타입의 content를 저장한 tensor를 반환한다.  
 

#####  `tf.io.decode_png()`

image content가 담긴 tensor를 인자로 받아 decoding하여 uint8인 np.array로 바꾸어준다.   

> 이 때, uint8은 2의 8승이 256이기 때문에 0-255의 int의 값을 말한다.


##### `tf.string.split()`

image의 path를 통해서 레이블을 설정하기 위해 path를 쪼개기 위해 사용한다.  
이 때, path는 'string' type의 tensor이기 때문에 일반적으로 string의 split을 사용이 불가능하기 때문에 사용한다.


##### `tf.cast()`

tensor의 dtype을 변경하기 위해서 사용한다.   
numpy의 array의 경우에는 astype()을 사용할 수 있으나 tensor이기 떄문에 대신하여 사용한다.  

##### `cardinality()`

'차수'라는 개념인데, dataset에서 data의 갯수를 의미한다.
- 이 때, 중복의 유무에 따라 달라진다. 중복이 존재하는 경우 1개로 판단하기 때문이다.

# 이미지 저장

열심히 만든 이미지에 대해서 저장이 하고 싶을 수 있다. 이럴 때 저장하는 과정을 만들어두자.

- image resizing을 진행하고 나면 dtype이 float32이다. 해당 dtype으로는 image encoding이 불가하므로 uint8으로 변경해줘야한다.
- 가끔씩 이미지 전부가 검은색으로 나오는 경우가 있다. 이 경우는 0-255까지의 값을 가지는 uint8이 아니라 0-1의 값을 가지는 uint8이기 때문에 모든 픽셀값이 0에 가까워 어둡게 보이는 것이다.

## 구현방법

1. map을 통한 구현을 진행  
    > map() 의 경우 자체적으로 graph mode로 연산을 수행한다. 이 때, 연산의 최적화를 위해 모든 과정에 대한 vectorization 수행  
    > 또한 map은 dataset을 반환하는 작업으로, 전처리를 통해 새로운 dataset을 만들 때 사용한다.
    
2. image를 저장하는 작업의 경우, vectorization이 되지 않는 부분이 존재한다.
    > `unbatch()` 를 통해 반복 수행을 진행할 수 있도록 변환하여 for 문을 통해 원소별 수행
    > 이 작업은 매우 비효율적임. __(가급적 하지 않는 것이 좋다.)__
